![](https://www.qads.com.br/img/QADS.png)

#  Exercícios do módulo de Obtenção e Preparação de Dados

Remis Balaniuk, PhD

# Flat files

Obtenha os dados abertos das [viagens realizadas a serviço no Portal daTransparência](!http://www.portaltransparencia.gov.br/download-de-dados/viagens), processe as colunas normalizando atributos textuais e convertendo os demais para os tipos corretos e ao final salve o(s) dataframe(s) em disco usando o Pickle.

Download do csv do portal da transparência

In [229]:
import requests 
import zipfile

from google.colab import drive

def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

drive.mount('/content/drive')

%cd /content/drive/MyDrive/'Academia QADS'/'Obtendo e Preparando dados'/


if not os.path.exists("transparencia.zip"):
  download_url("http://www.portaltransparencia.gov.br/download-de-dados/viagens/2021", 'transparencia.zip' )

  with zipfile.ZipFile("transparencia.zip", 'r') as zip_ref:
      zip_ref.extract("2021_Viagem.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Academia QADS/Obtendo e Preparando dados


Leitura do arquivo de viagens

In [313]:
import pandas as pd
import unicodedata
import re
from typing import List

df = pd.read_csv("2021_Viagem.csv", sep=";", encoding="ISO8859-1")



def convert_to_snake_case_variable(input : List[str] ) -> List[str]:
  output = []
  for item in input: 
    item = re.sub(r'[\(\)\-]', "", item)
    item = re.sub(r'\s+', "_", item)
    item = unicodedata.normalize("NFKD", item.lower()).encode('ascii', errors='ignore').decode('utf-8')
    output.append(item)
  return output

df.columns = pd.Index(convert_to_snake_case_variable(df.columns.to_list()))

Exemplo dos dados a partir do topo do dataframe

In [314]:
df.head(1)

,identificador_do_processo_de_viagem,numero_da_proposta_pcdp,situacao,viagem_urgente,justificativa_urgencia_viagem,codigo_do_orgao_superior,nome_do_orgao_superior,codigo_orgao_solicitante,nome_orgao_solicitante,cpf_viajante,nome,cargo,funcao,descricao_funcao,periodo_data_de_inicio,periodo_data_de_fim,destinos,motivo,valor_diarias,valor_passagens,valor_outros_gastos
0,16829271,000163/21,Realizada,SIM,A viagem estva programada para ser realizda em...,36000,Ministério da Saúde,36211,Fundação Nacional de Saúde,***.980.641-**,VILIDIANA MORAES MOURA,ENGENHEIRO,FCT-03,Função Comissionada Técnica,03/02/2021,04/02/2021,Nova Brasilândia/MT,Visita técnica no municipio de Nova Brasilãndi...,"223,86","0,00","0,00"


Tipos de dados do dataframe

In [315]:
df.dtypes

identificador_do_processo_de_viagem     int64
numero_da_proposta_pcdp                object
situacao                               object
viagem_urgente                         object
justificativa_urgencia_viagem          object
codigo_do_orgao_superior                int64
nome_do_orgao_superior                 object
codigo_orgao_solicitante                int64
nome_orgao_solicitante                 object
cpf_viajante                           object
nome                                   object
cargo                                  object
funcao                                 object
descricao_funcao                       object
periodo_data_de_inicio                 object
periodo_data_de_fim                    object
destinos                               object
motivo                                 object
valor_diarias                          object
valor_passagens                        object
valor_outros_gastos                    object
dtype: object

Funções para auxiliar na verificação dos tipos de dados

In [316]:
def obtem_categoria(series):
    unique_count = series.nunique(dropna=False)
    total_count = len(series)
    if pd.api.types.is_numeric_dtype(series):
        return 'Numérico'
    elif pd.api.types.is_datetime64_dtype(series):
        return 'Data'
    elif unique_count==total_count:
        return 'Texto'
    else:
        return 'Categórico'

def print_categorias(df):
    for column_name in df.columns:
        print(column_name, ": ", obtem_categoria(df[column_name]))

def is_id_column(df, column):
  return df[column].shape[0] == df[column].nunique()

Tipos de dados no momento

In [317]:
print_categorias(df)

identificador_do_processo_de_viagem :  Numérico
numero_da_proposta_pcdp :  Categórico
situacao :  Categórico
viagem_urgente :  Categórico
justificativa_urgencia_viagem :  Categórico
codigo_do_orgao_superior :  Numérico
nome_do_orgao_superior :  Categórico
codigo_orgao_solicitante :  Numérico
nome_orgao_solicitante :  Categórico
cpf_viajante :  Categórico
nome :  Categórico
cargo :  Categórico
funcao :  Categórico
descricao_funcao :  Categórico
periodo_data_de_inicio :  Categórico
periodo_data_de_fim :  Categórico
destinos :  Categórico
motivo :  Categórico
valor_diarias :  Categórico
valor_passagens :  Categórico
valor_outros_gastos :  Categórico


In [361]:
import unicodedata

def normalize(string):
    return unicodedata.normalize("NFKD", str(string)).encode('ascii', errors='ignore').decode('utf-8')


In [362]:
df.identificador_do_processo_de_viagem.sample(5)

128238    17455561
98271     17407410
63454     17358078
53204     17343812
128500    17456000
Name: identificador_do_processo_de_viagem, dtype: int64

In [363]:
df.numero_da_proposta_pcdp.sample(5)

49805          000348/21
55796          000403/21
46990          008221/21
67732    Informacoes pro
72364          012825/21
Name: numero_da_proposta_pcdp, dtype: object

In [364]:
df.numero_da_proposta_pcdp =  df.numero_da_proposta_pcdp.apply(normalize)

In [365]:
df.situacao.sample(5)

20370         Realizada
2692          Realizada
34664         Realizada
97939     Nao realizada
121370        Realizada
Name: situacao, dtype: object

In [366]:
df.situacao = df.situacao.apply(normalize)

In [367]:
df.viagem_urgente.sample(5)

124079    NAO
140830    NAO
72800     SIM
37814     SIM
16067     SIM
Name: viagem_urgente, dtype: object

In [368]:
df.viagem_urgente = df.viagem_urgente.apply(normalize)

In [382]:
df.justificativa_urgencia_viagem.sample(5)

73231                            CONFORME ORDEM DE SERVICO.
135242                    SOMENTE NESTA DATA FOI SOLICITADA
21663     Informacao protegida por sigilo nos termos da ...
108273    Justificativa : Devido ao Decreto No 50.470 - ...
38296                                        Sem informacao
Name: justificativa_urgencia_viagem, dtype: object

In [385]:
df.justificativa_urgencia_viagem = df.justificativa_urgencia_viagem.apply(normalize)

In [389]:
df.codigo_do_orgao_superior.sample(5) 

2971      22000
102744    53000
65290     22000
140749    30000
22649     52000
Name: codigo_do_orgao_superior, dtype: int64

In [393]:
df.nome_do_orgao_superior.sample(5)

39293                  Ministério da Infraestrutura
101041    Ministério da Justiça e Segurança Pública
99610                        Ministério da Educação
54990                                Sem informação
138003    Ministério da Justiça e Segurança Pública
Name: nome_do_orgao_superior, dtype: object

In [395]:
df.nome_do_orgao_superior = df.nome_do_orgao_superior.apply(normalize)

In [399]:
df.codigo_orgao_solicitante.sample(5)

144029    22201
135096    22000
43711     37202
57675     52121
27391     64000
Name: codigo_orgao_solicitante, dtype: int64

In [402]:
df.nome_orgao_solicitante.sample(5)

104595                      Departamento de Polícia Federal
4236                        Departamento de Polícia Federal
99983                   Companhia Nacional de Abastecimento
36215     Ministério da Agricultura, Pecuária e Abasteci...
120393                                       Sem informação
Name: nome_orgao_solicitante, dtype: object

In [403]:
df.nome_orgao_solicitante = df.nome_orgao_solicitante.apply(normalize)

In [407]:
df.cpf_viajante.sample(5)

138057    ***.791.841-**
100284    ***.164.894-**
78215        ID008200135
100889    ***.988.056-**
143308    ***.108.262-**
Name: cpf_viajante, dtype: object

In [410]:
df.cpf_viajante = df.cpf_viajante.apply(normalize)

In [414]:
df.nome.sample(5)

136235               TIMOTEO AUGUSTO FISCHER
42386         JOAO GILBERTO SILVA CAVALCANTI
25419     RICHARDSON ALEX TORRES DE OLIVEIRA
69876                    RUAN MARIANO AGUIAR
131199     Informações protegidas por sigilo
Name: nome, dtype: object

In [417]:
df.nome = df.nome.apply(normalize)

In [420]:
df.cargo.sample(5)

144855    TECNICO REFORMA DESENVOLVIMENTO AGRARIO
23210                                         NaN
49853                             ENGENHEIRO-AREA
35428                                         NaN
40472           Informações protegidas por sigilo
Name: cargo, dtype: object

In [422]:
df.cargo = df.cargo.apply(normalize)

In [424]:
df.funcao.sample(5)

25818       DAS-1014
66903             -1
122015    OfIntermed
21778             -1
111513    OfSubalter
Name: funcao, dtype: object

In [427]:
df.funcao = df.funcao.apply(normalize)

In [429]:
df.descricao_funcao.sample(5)

73057                         Não Informado
106179    Informações protegidas por sigilo
103745                        Não Informado
112619        Cargo Comissionado de Direção
92316     Informações protegidas por sigilo
Name: descricao_funcao, dtype: object

In [431]:
df.descricao_funcao = df.descricao_funcao.apply(normalize)

In [436]:
df.periodo_data_de_inicio.head(5)

0    03/02/2021
1    11/05/2021
2    23/02/2021
3    24/04/2021
4    04/01/2021
Name: periodo_data_de_inicio, dtype: object

In [437]:
df.periodo_data_de_inicio = pd.to_datetime(df.periodo_data_de_inicio, format="%d/%m/%Y")

# Expressões regulares

Escreva as expressões regulares solicitadas. Cada exercício possui um conjunto de testes que deve ser completamente validado pela expressão regular. Cada teste possui registros onde expera-se um *match* da expressão regular, enquanto outros registros não devem apresentar nenhum *match*. Faça o exercício para que todos os registros do teste fiquem com a marca de correto (em verde). Para os exercícios, não utilize expressões regulares com mais de 80 caracteres.



Para esse exercício você precisará baixar o arquivo zipado "exerciciosRegex" disponibilizado na pasta QADS em (https://drive.google.com/drive/folders/1sUeC2VYx93ZXi3tKCJarUTVvq_7C2ECI?usp=sharing), descompactar esse arquivo e copiar todo o conteúdo em uma pasta do seu Google drive (contém um diretório chamado "exercicios" e um arquivo chamado re_jupyter.py). Em seguida você deve montar seu Google Drive aqui no Colab e acessar a pasta onde copiou esse conteúdo .

In [221]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [222]:
%cd /content/drive/MyDrive/'Academia QADS'/'Obtendo e Preparando dados'/exerciciosRegex/

/content/drive/MyDrive/Academia QADS/Obtendo e Preparando dados/exerciciosRegex


In [223]:
%ls

exercicios/  __pycache__/  re_jupyter.py


In [224]:
import imp
import re_jupyter
imp.reload(re_jupyter)

from re_jupyter import exercicio

Uma letra `a` imediatamente seguida por uma ou mais letras `b`

In [6]:
exercicio(1, r'(ab+)')

,ab
,abb
,abbbbbbbbb
,cbaaabb
,aba
,a (sem ocorrências)
,baa (sem ocorrências)
,ba (sem ocorrências)
,cba (sem ocorrências)
,cbaaa (sem ocorrências)
,ba (sem ocorrências)


Uma palavra de 6 letras que inicia com a letra `z`

In [7]:
exercicio(2, r'\bz\w{5}\b')

,zangas
,zangas espaço
,zéfiro
,segundo zéfiro
,zelo (sem ocorrências)
,zelos (sem ocorrências)
,zangada (sem ocorrências)
,zangados (sem ocorrências)
,azul (sem ocorrências)
,aziago (sem ocorrências)
,azougue (sem ocorrências)


Uma palavra com 8 a 10 letras que termine com a letra 'z'

In [8]:
exercicio(3, r'\b\w{7,9}z\b')

,robustez
,espaço robustez
,chafariz
,antes chafariz depois
,vetustez
,placidez
,insensatez
,gravidez
,invalidez
,luz (sem ocorrências)
,talvez (sem ocorrências)


3 dígitos consecutivos

In [9]:
exercicio(4, r'\d{3}')

,040 inicia com zero
,056 de novo
,100 cem
,789 789 789
,0101 binário
,000x termina em x
,ano de 1984
,data 01/01/2018
,zero 0 (sem ocorrências)
,dígito 1 (sem ocorrências)
,vinte 20 (sem ocorrências)


Um número inteiro com 2, 3 ou 4 dígitos, sendo que o primeiro dígito não pode ser zero

In [120]:
exercicio(5, r'\b[1-9]\d{1,3}\b')

,vinte 20
,100 cem
,789 789 789
,ano de 1984
,data 01/01/2018
,zero 0 (sem ocorrências)
,dígito 1 (sem ocorrências)
,inicia com zero 03 (sem ocorrências)
,03 também tem zero (sem ocorrências)
,040 inicia com zero (sem ocorrências)
,056 de novo (sem ocorrências)


Uma palavra de mais de dois caracteres que inicie com um dígito e termine com uma letra

In [228]:
exercicio(6, r'\b\d[\w\d]+\w\b')

,parágrafo 11b
,1111111b
,1bbbbbbb
,1b1b1b1b
,teste 1a (sem ocorrências)
,dígitos 113 total
,soma quando 3a (sem ocorrências)
,bb1bbbbb11b (sem ocorrências)
,b1b1b1b1 (sem ocorrências)
,111
,1a1


Uma palavra seguida de um espaço seguido de um sequência de um ou mais dígitos

In [12]:
exercicio(7, r'\w+\s\d+')

,teste 1234
,parágrafo 12345
,letra 1
,a b 1 c
,cc 11 a
,1 1
,dois espaços não pode 2 (sem ocorrências)


Um dia da semana: segunda-feira, terça-feira, quarta-feira, quinta-feira, sexta-feira. Adicionalmente, permita a escrita dos dias da semana com espaço no lugar do hífen ou com as palavras unidas (ex.: quartafeira e quinta feira).

In [13]:
exercicio(8, r'segunda[-\s]?feira|\bterça[-\s]?feira|quarta[-\s]?feira|quinta[-\s]?feira|^sexta[-\s]?feira')

,hoje é segunda-feira
,amanhã será terça feira
,quartafeira já passou
,quinta-feira foi dia 3
,sextafeira
,sábado feriado (sem ocorrências)
,domingo feriado (sem ocorrências)
,2a feira internacional (sem ocorrências)
,sexta_feira (sem ocorrências)
,quinta.feira (sem ocorrências)
,domingues (sem ocorrências)


Uma URL no formato: http://www.&lt;dominio&gt;.com.br/&lt;arquivo&gt;.html. O &lt;arquivo&gt; e o &lt;domínio&gt; somente podem contar letras de a a z (minúscula ou maiúsculas), dígitos e os sinais _ e -.

In [14]:
exercicio(9, r'http://www\.[\w\d_-]+.com\.br/[\w\d_-]+\.html')

,http://www.google.com.br/index.html
,http://www.terra.com.br/underline_1.html
,http://www.teste-novo.com.br/hiFen-1.html
,http://www.google.com/index.html (sem ocorrências)
,http://uol.com.br/teste.html (sem ocorrências)
,http://bol.com.br/nao/podem/existir/barras.html (sem ocorrências)
,https://www.google.com/https/também/nao/pode.html (sem ocorrências)
,http://www.teste_velha.com.br/111-html (sem ocorrências)
,http://www-testevelho.com-br/111.html (sem ocorrências)
,http://www..com-br/1.html (sem ocorrências)
,http://www.com-br/2.html (sem ocorrências)


Duas palavras unidas pelas preposições 'de', 'do', 'da', 'dos' ou 'das'. As palavras somente podem conter letras e elas são separadas por um ou mais espaços.

In [15]:
exercicio(10, r'\w+\s+d[eoa]s?\s+\w+')

,"Olhos do mundo,"
,"zelos do marido,"
,"morte do Viegas,"
,uma vertigem de cabriolas
,descendente do cavalo de Aquiles
,a minha teoria das edições humanas
,me pareceu entrar na região dos gelos eternos
,"Esta idéia, (sem ocorrências)"
,"rútila e grande, (sem ocorrências)"
,"trajada ao bizarro, (sem ocorrências)"
,"como diria o Padre Bernardes, (sem ocorrências)"


Uma palavra de no mínimo 3 letras e com TODAS as letras maiúsculas

In [17]:
exercicio(11, r'\b[A-ZÓÉ]{3,}\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empreiteiras ODEBRECHT
,integrantes do Grupo OAS
,empresa GFD INVESTIMENTOS
,conforme já apurado pelo TCU
,"no caso das empresas MO Consultoria, Empreiteira Rigidez e RCI Software"
,RELATÓRIO DE TENDÊNCIAs


Duas palavras consecutivas, de no mínimo 2 letras cada, com TODAS as letras maiúsculas

In [92]:
exercicio(12, r'\b[A-ZÁÓÉÊ]{2,}\s+[A-ZÁÓÉÊ]{2,}\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,VARIOS ESPACOS
,empreiteiras ODEBRECHT (sem ocorrências)
,integrantes do Grupo OAS (sem ocorrências)
,conforme já apurado pelo TCU (sem ocorrências)


Duas ou mais palavras consecutivas com todas as letras maiúsculas. Cada palavra pode possuir qualquer quantidade de letras.

In [185]:
exercicio(13, r'\b[A-ZÓÉ]+\s+[A-ZÁ]+\b(\s[A-ZÃÊ]+)?(\s[A-Z]+)?\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,TESTE A TESTE
,A BE C DE
,VARIOS ESPACOS
,empreiteiras ODEBRECHT (sem ocorrências)


Uma linha contendo exatamente um endereço [IP](https://en.wikipedia.org/wiki/IP_address). ex.: 127.0.0.1, 192.168.0.254. Um IP é formado por exatamente 4 números separados por pontos. Cada número possui no máximo 3 dígitos.

In [65]:
exercicio(14, r'^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$')

,127.0.0.1
,192.168.0.254
,10.11.10.11
,127.127.127.127
,9999.0.0.1 (sem ocorrências)
,127404041 (sem ocorrências)
,a127.0.0.1 (sem ocorrências)
,127.a.a.1 (sem ocorrências)
,127.0.0.1a (sem ocorrências)
,xxx.yyy.zzz.www (sem ocorrências)
,0.0.0.a (sem ocorrências)


Uma linha contendo exatamente um CPF com separador de milhares e hífen.

In [69]:
exercicio(15, r'^\d{3}\.\d{3}\.\d{3}\-\d{2}$')

,000.000.000-00
,111.111.111-11
,0111.111.111-11 (sem ocorrências)
,111.111.111-110 (sem ocorrências)
,111.0111.0111-11 (sem ocorrências)
,11101110111-11 (sem ocorrências)
,111.111.111111 (sem ocorrências)
,1110110111111 (sem ocorrências)
,111011011111 (sem ocorrências)
,11101101111 (sem ocorrências)
,1110110111 (sem ocorrências)


Uma linha contendo exatamente um CPF com separador de milhares opcional e hífen opcional.

In [70]:
exercicio(16, r'^\d{3}\.?\d{3}\.?\d{3}\-?\d{2}$')

,000.000.000-00
,111.111.111-11
,111111.111-11
,111.111111-11
,11101101111
,0111.111.111-11 (sem ocorrências)
,111.111.111-110 (sem ocorrências)
,111.0111.0111-11 (sem ocorrências)
,11101110111-11 (sem ocorrências)
,111.111.111111 (sem ocorrências)
,1110110111111 (sem ocorrências)


Uma linha contendo exatamente um CNPJ com separador de milhares, barra e hífen.

In [72]:
exercicio(17, r'^\d{2}\.\d{3}\.\d{3}\/\d{4}\-\d{2}$')

,00.000.000/0001-00
,xx.000.000/0001-xx (sem ocorrências)
,0011001000/0001-00 (sem ocorrências)
,00.000.00010001-00 (sem ocorrências)
,00.000.000/0001100 (sem ocorrências)
,00000.000/0001-00 (sem ocorrências)
,00000000/0001-00 (sem ocorrências)
,000000000001-00 (sem ocorrências)
,00000000000100 (sem ocorrências)
,x00000000000100 (sem ocorrências)
,00000000000100x (sem ocorrências)


Uma linha contendo exatamente um CNPJ com separador de milhares opcional, barra opcional e hífen opcional.

In [73]:
exercicio(18, r'^\d{2}\.?\d{3}\.?\d{3}\/?\d{4}\-?\d{2}$')

,00.000.000/0001-00
,00000.000/0001-00
,00000000/0001-00
,000000000001-00
,00000000000100
,xx.000.000/0001-xx (sem ocorrências)
,0011001000/0001-00 (sem ocorrências)
,00.000.00010001-00 (sem ocorrências)
,00.000.000/0001100 (sem ocorrências)
,x00000000000100 (sem ocorrências)
,00000000000100x (sem ocorrências)


Uma linha contendo exatamente um CEP com separador de milhares e hífen

In [84]:
exercicio(19, r'^\d{2}\.\d{3}\-\d{3}')

,70.000-000
,7a.aaa-aaa (sem ocorrências)
,7000000000 (sem ocorrências)
,70000-000 (sem ocorrências)
,700000000 (sem ocorrências)
,70000000 (sem ocorrências)
,7000000 (sem ocorrências)
,700000 (sem ocorrências)
,70000 (sem ocorrências)
,7000 (sem ocorrências)
,70000aaa (sem ocorrências)


Uma linha contendo exatamente um CEP com separador de milhares opcional e hífen opcional

In [83]:
exercicio(20, r'^\d{2}\.?\d{3}\-?\d{3}$')

,70.000-000
,70000-000
,70000000
,7a.aaa-aaa (sem ocorrências)
,7000000000 (sem ocorrências)
,700000000 (sem ocorrências)
,7000000 (sem ocorrências)
,700000 (sem ocorrências)
,70000 (sem ocorrências)
,7000 (sem ocorrências)
,70000aaa (sem ocorrências)


Uma linha contendo exatamente um telefone no formato (xx)xxxx-xxxx.

In [79]:
exercicio(21, r'^\(\d{2}\)\d{4}\-\d{4}$')

,(61)3316-0000
,3316-0000 (sem ocorrências)
,x61x3316x0000 (sem ocorrências)
,(61)3316x0000 (sem ocorrências)
,((61)3316-00006 (sem ocorrências)
,77(61)3316x000077 (sem ocorrências)
,(77)(61)3316-00007 (sem ocorrências)


Uma linha contendo exatamente um telefone no formato (xx)xxxx-xxxx ou xxxx-xxxx (sem DDD).

In [96]:
exercicio(22, r'^\(\d{2}\)\d{4}\-\d{4}|^\d{4}\-\d{4}')

,(61)3316-0000
,3316-0000
,x61x3316x0000 (sem ocorrências)
,(61)3316x0000 (sem ocorrências)
,((61)3316-00006 (sem ocorrências)
,77(61)3316x000077 (sem ocorrências)
,(77)(61)3316-00007 (sem ocorrências)


# JSON

Usando a API do Banco Central, descrita em https://www.bcb.gov.br/conteudo/dadosabertos/BCBDepin/gnastportal-dados-abertostaxas-de-cambio---todos-os-boletins-diarios.pdf, escreva uma função que apresente ao usuário as moedas disponíveis para cotação, peça a ele que escolha uma ou mais, peça que o usuário escolha uma data inicial e uma data final, obtenha as cotações para as moedas selecionadas no período, armazene num dataframe e ao final salve o dataframe no disco usando o Pickle.

# HTML, JSON e REGEX

Crie um extrator de contratos a partir da seção 3 do Diário Oficial da União, obtendo as informações da página (link e texto) e por meio de expressões regulares extraia o nome do contratante, CNPJ e nome do contratado. Coloque o resultado num dataframe e salve em disco com o Pickle.